In [85]:
import pandas as pd, numpy as np

In [86]:
# read in CSV version of HSR dataset

raw = pd.read_csv("FD25 - CAHSR - 2013_RPSP_Data_Public.csv", encoding = 'iso-8859-1')

C:\Users\pksohn\Anaconda\envs\ipykernel_py3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [87]:
raw.head()

,MODE,SYSTEM,UNID,SAQ,Interview Date,Month of Travel TXT,Year of Travel,System Location,Airline,AIRPORT,...,E6_HSRail_Frequency,E6_HSRail_Access,E6_HSRail_Egress,E6_Air_Travel_Time,E6_Air_Cost,E6_Air_Reliability,E6_Air_Frequency,E6_Air_Access,E6_Air_Egress,E6_EXPNO
0,AIR,SFO,10001,2,20130620,June,2013.0,1,3.0,8.0,...,25,45,40,95,90.0,95.0,20,40,50,44
1,AIR,SFO,10002,2,20130620,June,2013.0,1,3.0,8.0,...,25,25,50,80,65.0,95.0,30,30,40,38
2,AIR,SFO,10003,1,20130620,June,2013.0,1,3.0,8.0,...,35,30,60,95,145.0,70.0,30,40,40,20
3,AIR,SFO,10004,1,20130620,June,2013.0,1,3.0,8.0,...,20,30,30,110,115.0,95.0,30,50,40,50
4,AIR,SFO,10005,1,20130620,June,2013.0,1,3.0,8.0,...,20,30,30,110,115.0,95.0,30,50,40,50


In [88]:
# get SP variable names
vars_E6 = ["E6_Auto_Travel_Time", "E6_Auto_Cost", "E6_Auto_Reliability" ,"E6_HSRail_Travel_Time" , "E6_HSRail_Cost", 
           "E6_HSRail_Reliability", "E6_HSRail_Frequency", "E6_HSRail_Access", "E6_HSRail_Egress","E6_Air_Travel_Time", 
           "E6_Air_Cost", "E6_Air_Reliability", "E6_Air_Frequency", "E6_Air_Access", "E6_Air_Egress", "E6_EXPNO"]

# strip "E6_" from above variable names
SP = [w.replace("E6_","") for w in vars_E6]

# get the core variable names
base_vars = raw.columns[0:97].tolist()

# create an empty dataframe to append to
df = pd.DataFrame()

In [91]:
# this loop creates one row for each SP choice

for i in range(1,7):
    
    # get the string for each SP choice (e.g. "E6_")
    prefix = "E" + str(i) + "_"
    
    # get the SP variables for choice i
    iter_vars = [prefix + w for w in SP]
    
    # dataframe with only base variables and SP vars for choice i
    temp_df = raw[base_vars + iter_vars]
    
    # create a variable denoting which choice in each row
    temp_df["SP_exp"] = i
    
    # drop the prefixes
    temp_df = temp_df.rename(columns = {col: col.replace(prefix,"") for col in temp_df.columns})
    
    # append to master dataframe
    df = df.append(temp_df,ignore_index=True)

C:\Users\pksohn\Anaconda\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [111]:
# create a numeric approximation of income categories using midpoints

income_dict = {
    1: 2500,
    2: np.mean([5000,25000]),
    3: np.mean([25000,50000]),
    4: np.mean([50000,75000]),
    5: np.mean([75000,100000]),
    6: np.mean([100000,150000]),
    7: np.mean([150000,200000]),
    8: 250000,
    9: None,
    10: None,
    0: None
}

df["hh_income"] = df.apply(lambda row: income_dict[row["HHINC"]],axis=1)

# create per-capita income variable

df["per_cap_income"] = df.apply(lambda row: row["hh_income"] / row["HHSIZE"] if (row["HHSIZE"] < 7) &  (row["HHSIZE"] != 0) else None, axis = 1)

Formula for expenditure rate from "INTRODUCING THE EXPENDITURE RATE IN THE ESTIMATION OF MODE CHOICE MODELS", Jara-Diaz, S R and
De DOIS ORTUZAR, J, 1989:

$$
g = k * \frac{PCFI}{168 - W}
$$

Where g is expenditure rate, k is a constant to express g in certain units (here we will use $/hr), PCFI is per capita family income, W is working hours in a week, and 168 is the number of hours in a week.

In [131]:
# create expenditure rate variable

df["empstat_miss"] = df["EMPSTAT"]
df.empstat_miss[(df.empstat_miss == 7) | (df.empstat_miss == 8) | (df.empstat_miss == 9) | (df.empstat_miss == 0)] = None

workweek_dict = {
    1: 40,
    2: 20,
    3: 0,
    4: 40,
    5: 0,
    6: 40,
    7: 20
}

df["avail_time"] = df.apply(lambda row: 168 - workweek_dict[row["empstat_miss"]] if ~ np.isnan(row["empstat_miss"]) else None, axis = 1)
df["exp_rate"] = df.apply(lambda row: ((row["per_cap_income"] / 52) / row["avail_time"]) , axis = 1)


C:\Users\pksohn\Anaconda\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [132]:
df.exp_rate.describe()

count    20382.000000
mean         8.185387
std          6.476089
min          0.047695
25%          3.286508
50%          6.573017
75%         10.016026
max         37.560096
Name: exp_rate, dtype: float64

In [154]:
# Travel Cost variables

# CAR
df["CV_OVER_G_Car"] = (df.COST_GAS + df.TOLL_MAIN + df.COST_OTHAUTO) / df.exp_rate
df["CV2_OVER_GI_Car"] = (df["CV_OVER_G_Car"] ** 2) / (df.exp_rate * df.per_cap_income)
# income variable used is per-capita family income

# AIR RP
df["AIRFARE_num"] = df.AIRFARE
df.AIRFARE_num = pd.to_numeric(df.AIRFARE_num, errors= "coerce")
df["CV_OVER_G_Air_RP"] = df.AIRFARE_num.astype(float) + df.ACC_PRK + df.ACC_TOLL + df.ACC_FARE+ df.EGR_PRK + df.EGR_TOLL + df.EGR_FARE
df["CV2_OVER_GI_Air_RP"] = (df["CV_OVER_G_Air_RP"] ** 2) / (df.exp_rate * df.per_cap_income)

# RAIL
df["CV_OVER_G_HS_Rail"] = df.HSRail_Cost
df["CV2_OVER_GI_HS_Rail"] =  (df["CV_OVER_G_HS_Rail"] ** 2) / (df.exp_rate * df.per_cap_income)

# AIR SP
df["CV_OVER_G_Air_SP"] = df.Air_Cost
df["CV2_OVER_GI_Air_SP"] = (df["CV_OVER_G_Air_SP"] ** 2) / (df.exp_rate * df.per_cap_income)

In [160]:
# Convert categorical access time variables to numeric

access_capital_caltrain = {
    1: np.mean([0,5]),
    2: np.mean([5,9]),
    3: np.mean([10,14]),
    4: np.mean([15,19]),
    5: np.mean([20,29]),
    6: 40,
    7: None,
    0: None
}

access_metrolink = {
    1: np.mean([0,5]),
    2: np.mean([5,9]),
    3: np.mean([10,14]),
    4: np.mean([15,19]),
    5: np.mean([20,29]),
    6: np.mean([30,44]),
    7: np.mean([45,59]),
    8: 75,
    9: None,
    0: None
}

access_air = {
    1: np.mean([0,15]),
    2: np.mean([15,19]),
    3: np.mean([20,29]),
    4: np.mean([30,44]),
    5: np.mean([45,59]),
    6: np.mean([60,74]),
    7: np.mean([75,89]),
    8: np.mean([90,119]),
    9: np.mean([120,180]),
    10: 210,
    11: None,
    0: None
}

def access_time(row):
    if row["MODE"] == "AIR":
        return access_air[row["T2ACESS"]]
    elif (row["MODE"] == "RAIL") & ((row["SYSTEM"] == "CALTRAIN") | (row["SYSTEM"] == "CAPITOL CORRIDOR")):
        return access_capital_caltrain[row["T2ACESS"]]
    elif (row["MODE"] == "RAIL") & ((row["SYSTEM"] == "METROLINK") | (row["SYSTEM"] == "AMTRAK CENTRAL CA")):
        return access_metrolink[row["T2ACESS"]]

df["access_time"] = None
df["access_time"] = df.apply(access_time,axis=1)

In [163]:
# Convert categorical access time variables to numeric - PART II

fndes_rail = {
    1: np.mean([0,5]),
    2: np.mean([5,9]),
    3: np.mean([10,14]),
    4: np.mean([15,19]),
    5: np.mean([20,29]),
    6: 40,
    7: None,
    0: None
}


fndes_air = {
    1: np.mean([0,15]),
    2: np.mean([15,19]),
    3: np.mean([20,29]),
    4: np.mean([30,44]),
    5: np.mean([45,59]),
    6: np.mean([60,74]),
    7: np.mean([75,89]),
    8: np.mean([90,119]),
    9: np.mean([120,180]),
    10: 210,
    11: None,
    0: None
}

def fndes_time(row):
    if row["MODE"] == "AIR":
        return fndes_air[row["T2FNDES"]]
    elif row["MODE"] == "RAIL":
        return fndes_rail[row["T2FNDES"]]

df["fndes_time"] = None
df["fndes_time"] = df.apply(fndes_time,axis=1)

In [164]:
# Convert categorical board time variables to numeric


def board_time(row):
    if row["MODE"] == "AIR":
        return access_air[row["T2BRD"]]
    elif row["MODE"] == "RAIL":
        return access_capital_caltrain[row["T2BRD"]]

df["brd_time"] = None
df["brd_time"] = df.apply(board_time,axis=1)

In [165]:
df.to_csv("HSR_dataset.dat",sep='\t')

In [ ]:
# Filter dataset for LA/SF market

# NEed to discuss - dependent on mode